In [2]:
import pandas as pd
TrainingLabels = pd.read_csv('Datasets/Training labels.csv', index_col='id')
TrainingFeatures = pd.read_csv('Datasets/Training features.csv', index_col='id')
TestFeatures = pd.read_csv('Datasets/Test features.csv', index_col='id')



In [3]:
preprocess = pd.concat([TrainingFeatures,TestFeatures])

Dropping unnecessary columns

In [4]:
preprocess.drop(['wpt_name','num_private','region','recorded_by',
'extraction_type_class','extraction_type','payment_type','water_quality','quantity_group','source_type',
'source','waterpoint_type','date_recorded','management_group','scheme_name'],axis=1,inplace=True)
preprocess.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region_code', 'district_code',
       'lga', 'ward', 'population', 'public_meeting', 'scheme_management',
       'permit', 'construction_year', 'extraction_type_group', 'management',
       'payment', 'quality_group', 'quantity', 'source_class',
       'waterpoint_type_group'],
      dtype='object')

Cleaning Funder

In [5]:
import numpy as nan
preprocess['funder'] = preprocess['funder'].str.lower()
preprocess['funder'].fillna(preprocess.installer, inplace=True)
preprocess['funder'].fillna('nan', inplace=True)
preprocess['funder'].replace(to_replace=('0', '-', ''), value='nan', inplace=True)
funder_reduction = ['kkkt','vision','foundat','tlc','wsd','norad','sema','wfp','ded','unicef','danida','hes','rws','dhv','isf']

for category in funder_reduction:
    subCategory = preprocess[preprocess['funder'].str.contains(category)]
    unique = subCategory['funder'].unique()
    preprocess['funder'].replace(to_replace=unique,value=category, inplace=True)

multi = [
[['hifab','amref','oxfa','wateraid','Wate Aid','Water Aid','adra','plan','Ces(gmbh)','Ces (gmbh)','shipo','dmdd','oikos'],'project'],
[['mzee','priva','mwita','ltd','bank','kanis'],'private'],
[['municipal council', 'distric','govern','ministry','supply','adb','rudep'],'govern'],
[['rc','roman',' mission','cathoric','st ','churc','anglican','tcrs'],'church'],
[['total land care','total landcare','totaland care','totoland','totoland care'],'totallandcare'],
[['tassaf','tasaf'],'tasaf'],
[['ngo ',' ngo'],'ngo'],
[['rotary','club'],'club'],
[['islam','muslim'],'muslim'],
[['said ',' said'],'said'],
[['british','japan','egypt','belgian','italy','iran','china','finland','irish','swedish','german','netherlan','european','kuwait','african','jaica','jica'],'foreign'],
[['village','community','rural','halmashauri','lga'],'village'],
[['lga','adb'],'lga & adb'],
[['Fin Water','Fini Water','Finn Water','Finw','Finwater','Fiwater'],'fini']
]
for category in multi:
    for name in category[0]:
        subCategory = preprocess.loc[preprocess['funder'].str.contains(name)]
        unique = subCategory['funder'].unique()
        preprocess['funder'].replace(to_replace=unique,value=category[1], inplace=True)

preprocess['funder'].replace(to_replace=('mission','missio','missi'),value='church', inplace=True)


count = preprocess.groupby('funder')['funder'].count()
other_list = count[count<100].index.tolist()
preprocess['funder'].replace(to_replace=other_list, value='other', inplace=True)  

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(preprocess.groupby('funder')['funder'].count().sort_values(ascending=False))

C:\Users\Yasith\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


funder
govern                     15431
other                      10791
nan                         5419
danida                      3921
project                     3661
private                     3402
foreign                     3163
church                      3077
hes                         2802
kkkt                        1953
rws                         1742
vision                      1603
village                     1430
unicef                      1350
tasaf                       1316
dhv                         1146
norad                       1078
dwsp                        1012
water                        739
dwe                          605
fini water                   474
isf                          429
wsd                          322
ces(gmbh)                    316
finw                         281
dh                           261
ded                          246
kiliwater                    243
magadini-makiwaru water      226
w.b                          223
go 

Creating Management_cross

In [6]:
preprocess['management_cross'] = preprocess['scheme_management'] + preprocess['management']
count = preprocess.groupby('management_cross')['management_cross'].count()
other_list = count[count<100].index.tolist()
preprocess['management_cross'].replace(to_replace=other_list, value='other', inplace=True)  
preprocess.drop(['scheme_management','management'],axis=1,inplace=True)

Other preprocessing and feature engineering

In [7]:
import numpy as np

preprocess['gps_height'] = preprocess['gps_height'].abs()
preprocess['gps_height'].replace(to_replace=0, value=np.nan, inplace=True)

preprocess['longitude'].replace(to_replace=0, value=np.nan, inplace=True)

preprocess['construction_year'].replace(to_replace=0, value=np.nan, inplace=True)
preprocess['age'] = 2013 - preprocess.construction_year
preprocess.drop('construction_year', axis = 1, inplace = True)

preprocess.drop('installer', axis = 1, inplace = True)

bool_mapper = {"False":0, "True":1}
preprocess["permit"] = preprocess["permit"].replace(bool_mapper)
preprocess["public_meeting"] = preprocess["public_meeting"].replace(bool_mapper)


In [8]:
preprocess.isna().sum()

amount_tsh                   0
funder                       0
gps_height               25649
longitude                 2269
latitude                     0
basin                        0
subvillage                 470
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4155
permit                    3793
extraction_type_group        0
payment                      0
quality_group                0
quantity                     0
source_class                 0
waterpoint_type_group        0
management_cross          4846
age                      25969
dtype: int64

In [9]:
preprocess["permit"] = preprocess["permit"].astype('float')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('float')

Filling missing values

In [10]:
for fill in ['age','longitude','gps_height']:
    preprocess[fill].fillna(preprocess[fill].mean(), inplace=True)

for fill in ['public_meeting','permit']:
    preprocess[fill].fillna(preprocess[fill].mode()[0], inplace=True)

for fill in ['management_cross','subvillage']:
    preprocess[fill].fillna('nan',inplace=True)

In [11]:
preprocess["permit"] = preprocess["permit"].astype('uint8')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('uint8')

In [12]:
object_columns = preprocess.select_dtypes(include=['object']).columns
for object_col in object_columns:
    preprocess[object_col] = preprocess[object_col].astype('category')

In [13]:
preprocess.isna().sum()

amount_tsh               0
funder                   0
gps_height               0
longitude                0
latitude                 0
basin                    0
subvillage               0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
permit                   0
extraction_type_group    0
payment                  0
quality_group            0
quantity                 0
source_class             0
waterpoint_type_group    0
management_cross         0
age                      0
dtype: int64

In [14]:
TrainingFeatures = preprocess.iloc[:59400,:]
TestFeatures = preprocess.iloc[59400:,:]

TrainingFeatures.to_csv('Cleaned Datasets/training_cleaned.csv', index=True)
TestFeatures.to_csv('Cleaned Datasets/test_cleaned.csv', index=True)